In [1]:
!pip install pandas
!pip install pyspark
!pip install pyarrow


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
from pyspark.sql import SparkSession
 
# Building the SparkSession and name 
# it :'pandas to spark'
spark = SparkSession.builder.appName(
  "pandas to spark").getOrCreate()

spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")

24/10/28 22:22:26 WARN Utils: Your hostname, Maazs-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.2.24 instead (on interface en0)
24/10/28 22:22:26 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/28 22:22:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/10/28 22:22:27 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/10/28 22:22:27 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
24/10/28 22:22:27 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [3]:
import pandas as pd

data = [[1, '2019-02-17', '2019-02-28', 5], [1, '2019-03-01', '2019-03-22', 20], [2, '2019-02-01', '2019-02-20', 15], [2, '2019-02-21', '2019-03-31', 30]]
prices = pd.DataFrame(data, columns=['product_id', 'start_date', 'end_date', 'price']).astype({'product_id':'Int64', 'start_date':'datetime64[ns]', 'end_date':'datetime64[ns]', 'price':'Int64'})
data = [[1, '2019-02-25', 100], [1, '2019-03-01', 15], [2, '2019-02-10', 200], [2, '2019-03-22', 30]]
units_sold = pd.DataFrame(data, columns=['product_id', 'purchase_date', 'units']).astype({'product_id':'Int64', 'purchase_date':'datetime64[ns]', 'units':'Int64'})

In [4]:
prices = spark.createDataFrame(prices)
prices.show()

units_sold = spark.createDataFrame(units_sold)
units_sold.show()

+----------+-------------------+-------------------+-----+
|product_id|         start_date|           end_date|price|
+----------+-------------------+-------------------+-----+
|         1|2019-02-17 00:00:00|2019-02-28 00:00:00|    5|
|         1|2019-03-01 00:00:00|2019-03-22 00:00:00|   20|
|         2|2019-02-01 00:00:00|2019-02-20 00:00:00|   15|
|         2|2019-02-21 00:00:00|2019-03-31 00:00:00|   30|
+----------+-------------------+-------------------+-----+

+----------+-------------------+-----+
|product_id|      purchase_date|units|
+----------+-------------------+-----+
|         1|2019-02-25 00:00:00|  100|
|         1|2019-03-01 00:00:00|   15|
|         2|2019-02-10 00:00:00|  200|
|         2|2019-03-22 00:00:00|   30|
+----------+-------------------+-----+



In [15]:
from pyspark.sql.functions import col, round, sum

prices \
    .join(
        units_sold,
        'product_id',
        'inner'
    ) \
.where('start_date <= purchase_date and end_date >= purchase_date') \
.withColumn('total_amount', col('price') * col('units')) \
.groupby('product_id') \
.agg(sum('units').alias('number_of_products_sold'), 
     sum('total_amount').alias('total_price_of_product')) \
.withColumn('average_price', 
            round(col('total_price_of_product') / col('number_of_products_sold'), 2)) \
.select(['product_id', 'average_price']) \
.show()


+----------+-------------+
|product_id|average_price|
+----------+-------------+
|         1|         6.96|
|         2|        16.96|
+----------+-------------+

